In [14]:
import transformers
import datasets
import torch
import logging

In [2]:
# Preprocessing
sep_token = '<sep>'
special_token = '<ANSWER>' # between context and answer
dataset_name = "squad" # id,title,context,question,answer{text,answer_start}
models_dir = "saved_models/t5_base_answer-aware-squad_mod"
checkpoint = 't5-base'
max_input_length = 512
max_target_length = 64

## Training
learning_rate = 1e-4
num_epochs = 7

In [16]:
dataset = datasets.load_dataset(dataset_name)

Found cached dataset squad (C:/Users/manuv/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00, 199.96it/s]


In [17]:
dataset["train"][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [18]:
model = transformers.T5ForConditionalGeneration.from_pretrained(checkpoint)
tokenizer = transformers.T5TokenizerFast.from_pretrained(checkpoint)

c:\Users\manuv\miniconda3\envs\t5_qg\Lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [19]:
tokenizer.sep_token = sep_token
tokenizer.add_tokens([sep_token])
tokenizer.add_tokens([special_token])
tokenizer.special_tokens_map.update({special_token: '[unused100]'})
model.resize_token_embeddings(len(tokenizer))

Embedding(32102, 768)

In [68]:

# Tokenize examples
def convert_to_features(example_batch):

    input_encodings = tokenizer.batch_encode_plus(example_batch["input"], 
                                                  max_length=max_input_length, 
                                                  add_special_tokens=True,
                                                  truncation=True, 
                                                  pad_to_max_length=True)
    
    target_encodings = tokenizer.batch_encode_plus(example_batch["question"], 
                                                   max_length=max_target_length, 
                                                   add_special_tokens=True,
                                                   truncation=True, pad_to_max_length=True)
                                                   
    encodings = {
        'input_ids': input_encodings['input_ids'], 
        'attention_mask': input_encodings['attention_mask'],
        'decoder_input_ids': target_encodings['input_ids'],
        'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings

def add_eos_examples(example):
    example['input'] = example['context'] + " " + special_token + example["answers"]["text"][0]
    example['question'] = example['question'] + " " + sep_token
    return example


def add_special_tokens(example):
  example['question'] = example['question'].replace("{sep_token}", sep_token)
  return example

In [69]:
tokenized_dataset  = dataset.map(add_eos_examples)
tokenized_dataset = tokenized_dataset.map(add_special_tokens)
tokenized_dataset  = tokenized_dataset.map(convert_to_features,  batched=True)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]               c:\Users\manuv\miniconda3\envs\t5_qg\Lib\site-packages\transformers\tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [74]:
tokenized_dataset["train"][0]["input"]

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary. <ANSWER>Saint Bernadette Soubirous'

In [75]:
tokenized_dataset = tokenized_dataset.remove_columns(
    ["input", "question","context","answers"]
)

train_dataset = tokenized_dataset["train"]
valid_dataset = tokenized_dataset["validation"]

columns = ['input_ids', 'decoder_input_ids', 'attention_mask', 'decoder_attention_mask']
train_dataset.set_format(type='torch', columns=columns)
valid_dataset.set_format(type='torch', columns=columns)

In [76]:
torch.save(train_dataset, 'train_data.pt')
torch.save(valid_dataset, 'valid_data.pt')

In [77]:
from typing import Dict, List

class T2TDataCollator():
    def __call__(self, batch: List) -> Dict[str, torch.Tensor]:
        """
        Take a list of samples from a Dataset and collate them into a batch.
        Returns:
        A dictionary of tensors
        """

        input_ids = torch.stack([example['input_ids'] for example in batch])
        lm_labels = torch.stack([example['decoder_input_ids'] for example in batch])
        lm_labels[lm_labels[:, :] == 0] = -100 
        attention_mask = torch.stack([example['attention_mask'] for example in batch])
        decoder_attention_mask = torch.stack([example['decoder_attention_mask'] for example in batch])

        return {
            'input_ids': input_ids, 
            'attention_mask': attention_mask,
            'labels': lm_labels, 
            'decoder_attention_mask': decoder_attention_mask
        }

In [78]:
training_args = transformers.TrainingArguments(output_dir=models_dir, 
                                  per_device_train_batch_size=4, 
                                  per_device_eval_batch_size=4,
                                  gradient_accumulation_steps=16,
                                  learning_rate=learning_rate, 
                                  num_train_epochs=num_epochs,
                                  logging_steps=100,
                                  run_name="t5_answer-aware_qg_squad",
                                  evaluation_strategy="steps",
                                  save_steps=500)

In [79]:
logger = logging.getLogger(__name__)

# Initialize our Trainer
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=T2TDataCollator()
)


In [80]:
if model.device.type == 'cuda':
    print('Model is on GPU')
else:
    print('Model is on CPU')

Model is on GPU


In [81]:
trainer.train()

c:\Users\manuv\miniconda3\envs\t5_qg\Lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  1%|          | 100/9576 [09:41<15:05:16,  5.73s/it]

{'loss': 2.4532, 'learning_rate': 9.895572263993317e-05, 'epoch': 0.07}


                                                     
  1%|          | 100/9576 [14:55<15:05:16,  5.73s/it]

{'eval_loss': 1.7000625133514404, 'eval_runtime': 313.7948, 'eval_samples_per_second': 33.684, 'eval_steps_per_second': 8.423, 'epoch': 0.07}


  2%|▏         | 200/9576 [24:28<14:56:34,  5.74s/it] 

...

                                                      
 99%|█████████▉| 9500/9576 [23:48:54<07:15,  5.73s/it]

{'eval_loss': 1.2983264923095703, 'eval_runtime': 314.7828, 'eval_samples_per_second': 33.579, 'eval_steps_per_second': 8.396, 'epoch': 6.94}


100%|██████████| 9576/9576 [23:57:23<00:00,  9.01s/it]   

{'train_runtime': 86243.8374, 'train_samples_per_second': 7.11, 'train_steps_per_second': 0.111, 'train_loss': 1.1823892386236485, 'epoch': 7.0}


TrainOutput(global_step=9576, training_loss=1.1823892386236485, metrics={'train_runtime': 86243.8374, 'train_samples_per_second': 7.11, 'train_steps_per_second': 0.111, 'train_loss': 1.1823892386236485, 'epoch': 7.0})

In [82]:
# save the model
trainer.save_model(models_dir)

In [3]:
from transformers import T5ForConditionalGeneration, T5TokenizerFast
# load the saved model
loaded_model = T5ForConditionalGeneration.from_pretrained(models_dir)

In [4]:
tokenizer = T5TokenizerFast.from_pretrained(checkpoint)

c:\Users\manuv\miniconda3\envs\t5_qg\Lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [5]:
def run_model(input_string, **generator_args):
    generator_args = {
    "max_length": 512,
    "num_beams": 4,# bij grotere num_beams is trager maar complexere vragen(niet per se betere vragen)
    "length_penalty": 1.5,
    "no_repeat_ngram_size": 3,
    "early_stopping": True,
    }
    input_string = "generate questions: " + input_string + " </s>"
    input_ids = tokenizer.encode(input_string, return_tensors="pt")
    res = loaded_model.generate(input_ids, **generator_args)
    output = tokenizer.batch_decode(res, skip_special_tokens=True)
    output = [item.split("<sep>") for item in output]
    return output

In [9]:
# context <ANSWER>answer
context = """
Cheese is an ancient food whose origins predate recorded history. There is no conclusive evidence indicating where cheesemaking originated, whether in Europe, Central Asia or the Middle East. Earliest proposed dates for the origin of cheesemaking range from around 8000 BCE, when sheep were first domesticated. Since animal skins and inflated internal organs have, since ancient times, provided storage vessels for a range of foodstuffs, it is probable that the process of cheese making was discovered accidentally by storing milk in a container made from the stomach of an animal, resulting in the milk being turned to curd and whey by the rennet from the stomach.[7] There is a legend—with variations—about the discovery of cheese by an Arab trader who used this method of storing milk.[8]

The earliest evidence of cheesemaking in the archaeological record dates back to 5500 BCE and is found in what is now Kuyavia, Poland, where strainers coated with milk-fat molecules have been found.[9]

Cheesemaking may have begun independently of this by the pressing and salting of curdled milk to preserve it. Observation that the effect of making cheese in an animal stomach gave more solid and better-textured curds may have led to the deliberate addition of rennet. Early archeological evidence of Egyptian cheese has been found in Egyptian tomb murals, dating to about 2000 BCE.[10] A 2018 scientific paper stated that the world's oldest cheese, dating to approximately 1200 BCE (3200 years before present), was found in ancient Egyptian tombs.[11][12]

The earliest cheeses were likely quite sour and salty, similar in texture to rustic cottage cheese or feta, a crumbly, flavorful Greek cheese. Cheese produced in Europe, where climates are cooler than the Middle East, required less salt for preservation. With less salt and acidity, the cheese became a suitable environment for useful microbes and molds, giving aged cheeses their respective flavors. The earliest ever discovered preserved cheese was found in the Taklamakan Desert in Xinjiang, China, dating back as early as 1615 BCE (3600 years before present). <ANSWER>ancient Egyptian tombs
"""

context_gf = """
#### Types of Bias

**Selection bias** is the tendency to skew your choice of data sources to 
those that are easily available, convenient, and/or cost-effective. As a 
result of this a bias is introduced by the selection of individuals, groups 
or data for analysis in such a way that proper randomization is not achieved, 
thereby ensuring that the sample obtained is not representative of the 
population intended to be analyzed. 
[Learn more about selection bias](https://en.wikipedia.org/wiki/Selection_bias).

**Self-selection bias** is a form of selection bias where you get the data 
from sources that “volunteered” to provide it. Most poll data has this type 
of bias. [Learn more about self-selection bias](https://en.wikipedia.org/wiki/Self-selection_bias)

**Omitted-variable bias** happens when your featurized data doesn't have a 
feature necessary for accurate prediction. For example, let's assume that you 
are working on a churn prediction model and you want to predict whether a 
customer cancels their subscription within six months. You train a model, and 
it's accurate enough; however, several weeks after deployment you see many 
unexpected false negatives. You investigate the decreased model performance 
and discover a new competitor now offers a very similar service for a lower 
price. This feature wasn't initially available to your model, therefore 
important information for accurate prediction was missing. 
[Learn more about omitted variable Bias](https://en.wikipedia.org/wiki/Omitted-variable_bias). <ANSWER>a form of selection bias
"""

run_model(context_gf)

[['What is self-selection bias?']]

In [88]:
# final metrics
trainer.evaluate()

100%|██████████| 2643/2643 [05:22<00:00,  8.21it/s]


{'eval_loss': 1.298403263092041,
 'eval_runtime': 322.0432,
 'eval_samples_per_second': 32.822,
 'eval_steps_per_second': 8.207,
 'epoch': 7.0}